# Recurrent Neural Network - LSTM


## SALVAR CONTEÚDO NO DRIVE PESSOAL!!!!! BAIXAR ESTA PASTA E FAZER UPLOAD PARA O SEU DRIVE

## Mount do drive pessoal

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import os
workdir_path = '/content/drive/My Drive/Aula7_LSTM'  # Inserir o local da pasta onde estão os arquivos de entrada (treino e teste)
os.chdir(workdir_path)

In [0]:
#Verificar se a pasta mapeada contem os arquivos abaixo
!ls

## Parte 1 - Preprocessamento de Dados

### Importar as bibliotecas

In [0]:
%tensorflow_version 1.x
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

### Importar a base de dados

In [0]:
dataset_train = pd.read_csv('train.csv')
dataset_train.head()

In [0]:
training_set = dataset_train.iloc[:, 1:2].values #só coluna da série em si

### gráfico da série temporal

In [0]:
plt.plot(training_set)
plt.xlabel("tempo")
plt.ylabel("Número de passageiros x10^3")
plt.title("Passageiros de avião")
plt.show()

### Normalização

In [0]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [0]:
#nao normalizado
training_set[0:5]

In [0]:
# normalizado
training_set_scaled[0:5]

In [0]:
len(training_set_scaled)

### Criar a estrutura de dados com janela 10 e output 1

In [0]:
window = 10
trainSize = len(training_set_scaled)
X_train = []
y_train = []
for i in range(window, trainSize):
    X_train.append(training_set_scaled[i-window:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

In [0]:
# inputs
pd.DataFrame(X_train).head()

In [0]:
pd.DataFrame(X_train).shape

In [0]:
#outputs
pd.DataFrame(y_train).head()

### Reshaping

In [0]:
X_train.shape

In [0]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [0]:
X_train.shape

## Parte 2 - Construção da RNN

### Bibliotecas necessárias

In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

### Construção da Rede completa

In [0]:
# Inicializar a RNN
regressor = Sequential()

# Adicionar a primeira camada LSTM e Dropout 
regressor.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
regressor.add(Dropout(0.2))
 
# Adicionar a segunda camada LSTM e Dropout
regressor.add(LSTM(units = 80, return_sequences = True))
regressor.add(Dropout(0.2))
 
# Adicionar a terceira camada LSTM e Dropout
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))
 
# camada de saída
regressor.add(Dense(units = 1))
 
# Compilar a rede
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Visualizar a rede
regressor.summary()

In [0]:
from keras import utils
utils.plot_model(regressor)

### Treinamento

In [0]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode= 'min', patience = 100)
history = regressor.fit(X_train, y_train, epochs = 250, batch_size = 32, 
                        validation_split=0.2, callbacks=[es])

In [0]:
plt.plot(history.history['loss'], label='train');
plt.plot(history.history['val_loss'], label='validation');
plt.legend();

## Parte 3 - Fazer as previsões e analisar os resultados

### Base de teste

In [0]:
dataset_test = pd.read_csv('test.csv')
test_airline = dataset_test.iloc[:, 1:2].values

In [0]:
len(dataset_test)

### Construir a estrutura para teste

In [0]:
dataset_total = pd.concat((dataset_train.iloc[:,1], dataset_test.iloc[:,1]), axis = 0)
testLength = len(dataset_test)
inputs = dataset_total[len(dataset_total) - testLength - window:].values
inputs = inputs.reshape(inputs.shape[0],1)
inputs.shape

In [0]:
inputs = sc.transform(inputs)
X_test = []
for i in range(window, inputs.shape[0]): 
    X_test.append(inputs[i-window:i, 0])
X_test = np.array(X_test)

X_test.shape

In [0]:
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
X_test.shape

### Fazer as previsões

In [0]:
predicted_airline = regressor.predict(X_test)
predicted_airline[0:5]

In [0]:
predicted_airline = sc.inverse_transform(predicted_airline)
predicted_airline[0:5]

### Visualizar os resultados de treino e teste

In [0]:
allTargetData = np.vstack((training_set, test_airline))
training_predicted_airline = regressor.predict(X_train)
training_predicted_airline = sc.inverse_transform(training_predicted_airline)
allForecastedData = np.vstack((training_set[0:window], training_predicted_airline, predicted_airline))
plt.plot(allTargetData, color = 'red', label = 'Real')
plt.plot(allForecastedData, color = 'blue', label = 'Previsto')
plt.title('Previsão de série temporal')
plt.xlabel('Tempo')
plt.ylabel('Passageiros')
plt.legend()
plt.savefig('predictions_training_test.png')
plt.show()

## Parte 4 - Métricas de avaliação

In [0]:
import math
from sklearn.metrics import mean_squared_error
rmse = math.sqrt(mean_squared_error(test_airline, predicted_airline))
print('RMSE: ', rmse)

In [0]:
mse = mean_squared_error(test_airline, predicted_airline)
print('MSE: ',mse)

In [0]:
mape = np.mean(np.abs((test_airline - predicted_airline) / test_airline)) * 100
print('MAPE: ',mape)